# 03 - Feature Engineering

Objective:
Transform event-level logs into behavioral user-level features.

Planned Features:

Access Behavior:
- Average access volume
- Resource diversity
- Action diversity

Temporal Behavior:
- Night access ratio
- Off-hours percentage
- Weekly activity trends

Stability Metrics:
- Access volume standard deviation
- Week-over-week change